## What I did here 
1. set working directory
2. save all necessary files in WD (Daegu map, occurrence map, Landsat band data ....)
3. calculate NDVI from Landsat national data
4. extract the NDVI at occurrence location 
5.  Check occurrence count ( threshold more than 30: some researcher suggest 25 for for logistic regression )
6.  call logistic regression model
7. fit the occurrence with NDVI (we have just 38 data points may under fit)
8. predict national occurrence map with NDVI 
9. crop for Daegu area 
10. Save raster data 

repeat for random presence absence data  from 4 to 10;
repeat for random presence absence data from 4 to 10, splitting train test data and .piloting ROC curve;
A simple logistic regression code 

In [1]:
import os
import shutil
import platform

def delete_temporary_files():
    # Windows specific temporary files
    if platform.system() == 'Windows':
        try:
            os.system('del /q/f/s %TEMP%\*')
        except Exception as e:
            print(f"Error deleting temporary files on Windows: {e}")
    # Unix-like temporary files
    else:
        try:
            shutil.rmtree('/tmp')
        except Exception as e:
            print(f"Error deleting temporary files on Unix-like system: {e}")

def delete_downloads():
    downloads_folder = os.path.expanduser('~') + '/Downloads'
    try:
        for filename in os.listdir(downloads_folder):
            file_path = os.path.join(downloads_folder, filename)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Error deleting {file_path} : {e}")
    except Exception as e:
        print(f"Error accessing Downloads folder: {e}")

def uninstall_unused_applications():
    # Add logic to uninstall unused applications here
    print("Uninstall unused applications function placeholder")

def clear_browser_cache():
    # Add logic to clear browser cache here
    print("Clear browser cache function placeholder")

def empty_recycle_bin_trash():
    # Add logic to empty Recycle Bin or Trash here
    print("Empty Recycle Bin or Trash function placeholder")

def free_up_space():
    print("Starting to free up space...")
    delete_temporary_files()
    delete_downloads()
    uninstall_unused_applications()
    clear_browser_cache()
    empty_recycle_bin_trash()
    print("Space cleanup completed.")

# Run the function to free up space
free_up_space()


Starting to free up space...
Uninstall unused applications function placeholder
Clear browser cache function placeholder
Empty Recycle Bin or Trash function placeholder
Space cleanup completed.


In [3]:
import os

# Change the working directory
os.chdir('C:/Users/IT/Downloads/2023')

# Verify the current working directory
print("Current working directory:", os.getcwd())


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:/Users/IT/Downloads/2023'

In [ ]:
# ls

In [ ]:
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from rasterio import features

In [ ]:
ndvi_meta['crs']

In [ ]:
# Paths to the Landsat 9 NIR (Band 5) and Red (Band 4) bands
nir_band_path = 'LC09_L1TP_114035_20230805_20230805_02_T1_B5.TIF'
red_band_path = 'LC09_L1TP_114035_20230805_20230805_02_T1_B4.TIF'

# Open the NIR band (Band 5)
with rio.open(nir_band_path) as nir_src:
    nir = nir_src.read(1).astype('float32')
    nir_meta = nir_src.meta

# Open the Red band (Band 4)
with rio.open(red_band_path) as red_src:
    red = red_src.read(1).astype('float32')

# Calculate NDVI
ndvi = (nir - red) / (nir + red)

# Update metadata to reflect the number of layers
ndvi_meta = nir_meta
ndvi_meta.update({"count": 1})

# # Save the NDVI image
# ndvi_path = 'NDVI.TIF' # we will use this data to crop for daegu are too 
# with rio.open(ndvi_path, 'w', **ndvi_meta) as dst:
#     dst.write(ndvi, 1)

# Plot NDVI
plt.imshow(ndvi, cmap='viridis', vmin=-1, vmax=1)
plt.colorbar(label='NDVI')
plt.title('NDVI from Landsat 9')
plt.show()

In [ ]:
nir_meta['crs']

In [ ]:
#Load the Shapefile and Extract NDVI Values

import geopandas as gpd
from shapely.geometry import Point

# Path to the shapefile containing examined points
shapefile_path = 'Daegu_point_korea_127/Daegu_point_korea_127.dbf'

# Read the shapefile using GeoPandas
gdf_points = gpd.read_file(shapefile_path)

# Reproject points to match the NDVI image CRS if necessary
gdf_points = gdf_points.to_crs(ndvi_meta['crs'])   

# Extract NDVI values at examined points
ndvi_values = []
for point in gdf_points.geometry:
    row, col = ~nir_meta['transform'] * (point.x, point.y)
    row, col = int(row), int(col)
    if 0 <= row < ndvi.shape[0] and 0 <= col < ndvi.shape[1]:
        ndvi_values.append(ndvi[row, col])
    else:
        ndvi_values.append(np.nan)

# Add NDVI values to GeoDataFrame
gdf_points['ndvi'] = ndvi_values

# # Print examined data points with their NDVI values
# print(gdf_points)
# gdf_points.plot()


In [ ]:
# we have data in pandas data frame we can fit different models setting predictor and responses 
gdf_points.head(3)



In [ ]:
#simply check the occurrences frequency
#normally need more than 30 points

sum(gdf_points['P__bengale']),sum(gdf_points['P__volans']),sum(gdf_points['N__caudatu'])

In [ ]:
# we found there is only oe species that cross the thrreshold 
#o we use P__bengale data
# Prepare data for logistic regression
X = gdf_points[['ndvi']].dropna()  # Predictor
y = gdf_points['P__bengale'][gdf_points['ndvi'].notna()]  # Response
model = LogisticRegression()
model.fit(X, y)

In [ ]:
model_ln = LinearRegression()
model_ln.fit(X, y)

In [ ]:
# Predict probabilities over the entire area (logistic)
ndvi_flat = ndvi.flatten()
ndvi_flat = ndvi_flat[~np.isnan(ndvi_flat)]  # Drop NaN values if any
probs = model.predict_proba(ndvi_flat.reshape(-1, 1))[:, 1]
# Reshape probabilities to match the NDVI shape
prob_map = np.full_like(ndvi, fill_value=np.nan)
prob_map.ravel()[~np.isnan(ndvi.ravel())] = probs
# Plot probability map
plt.figure(figsize=(10, 8))
plt.imshow(prob_map, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Predictive Map of Probability')
plt.show()

In [ ]:
# Predict probabilities over the entire area (Linear)
# Flatten the NDVI array
ndvi_flat = ndvi.flatten()
# Remove NaN values
ndvi_flat = ndvi_flat[~np.isnan(ndvi_flat)]
# Predict using the linear regression model
predictions = model_ln.predict(ndvi_flat.reshape(-1, 1))
# Convert the predictions to probabilities using the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
probs = sigmoid(predictions)
# Create a probability map with the same shape as the original NDVI array
prob_map = np.full_like(ndvi, fill_value=np.nan)
# Assign the predicted probabilities to the corresponding positions in the probability map
prob_map.ravel()[~np.isnan(ndvi.ravel())] = probs
# Plot the probability map
plt.figure(figsize=(10, 8))
plt.imshow(prob_map, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Predictive Map of Probability')
plt.show()

In [ ]:
# we see there are only presence data so if there is ndvi value, the is high probability 
#Let's make some absance area and fit again 

In [ ]:
import random
gdf_points['P_A_species']= [random.randint(0, 1) for _ in range(gdf_points.shape[0])]  # presence absance data 
sum(gdf_points['P_A_species'])

In [ ]:
X = gdf_points[['ndvi']].dropna()  # Predictor
y = gdf_points['P_A_species'][gdf_points['ndvi'].notna()]  # Response
model = LogisticRegression()
model.fit(X, y)

In [ ]:
# Predict probabilities over the entire area
ndvi_flat = ndvi.flatten()
ndvi_flat = ndvi_flat[~np.isnan(ndvi_flat)]  # Drop NaN values if any
probs = model.predict_proba(ndvi_flat.reshape(-1, 1))[:, 1]
# Reshape probabilities to match the NDVI shape
prob_map = np.full_like(ndvi, fill_value=np.nan)
prob_map.ravel()[~np.isnan(ndvi.ravel())] = probs
# Plot probability map
plt.figure(figsize=(10, 8))
plt.imshow(prob_map, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Predictive logistic Map of Probability')
plt.show()

In [ ]:

#  let's crop Daegu area
import geopandas as gpd
import rasterio
from rasterio import features
from rasterio.transform import from_bounds
from rasterio.crs import CRS

# Step 1: Read the vector file and convert to the desired CRS
daegu_boundary = gpd.read_file("Daegu/Daegu.dbf")
nir_meta = {'crs': ndvi_meta['crs']}  # Example CRS, replace with your actual CRS

daegu_boundary = daegu_boundary.to_crs(nir_meta['crs'])

# Step 2: Define the bounds and resolution for the raster
# Assuming you have bounds and resolution from some source (replace with actual values)
# Example bounds and resolution
bounds = (daegu_boundary.total_bounds[0], daegu_boundary.total_bounds[1], daegu_boundary.total_bounds[2], daegu_boundary.total_bounds[3])
resolution = (0.001, 0.001)  # Example resolution in degrees

# Calculate the width and height of the raster
width = int((bounds[2] - bounds[0]) / resolution[0])
height = int((bounds[3] - bounds[1]) / resolution[1])

# Step 3: Create a raster template
transform = from_bounds(bounds[0], bounds[1], bounds[2], bounds[3], width, height)
raster_template = rasterio.open('raster_template.tif', 'w', driver='GTiff', 
                                width=width, height=height, count=1, 
                                dtype=rasterio.uint8, 
                                crs=nir_meta['crs'], 
                                transform=transform)

# Step 4: Rasterize the vector data into the raster template
shapes = [(geom, 255) for geom in daegu_boundary.geometry]
rasterized = features.rasterize(shapes, out_shape=(height, width), transform=transform,
                                fill=0, default_value=255, dtype=rasterio.uint8)

# # Write rasterized data to the raster template
# raster_template.write(rasterized, 1)

# Close the raster template
# raster_template.close()

In [ ]:
# Load Daegu boundary shapefile (replace with  actual file path)
daegu_boundary = gpd.read_file("Daegu/Daegu.dbf").to_crs(nir_meta['crs']) 
# Plotting the boundary of Daegu
fig, ax = plt.subplots(figsize=(10, 8))  # Optional: Adjust figure size
daegu_boundary.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)

# Plotting the points on the same plot
gdf_points.plot(ax=ax, color='red', markersize=5)

# Adding titles and labels
ax.set_title('Daegu Boundary and Points')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Display the plot
plt.show()

In [ ]:
rasterized.shape

In [ ]:
prob_map.shape

In [ ]:
daegu_boundary.total_bounds

In [ ]:
# Rasterize the cropped Daegu boundary to use it as a mask
daegu_mask = features.rasterize(
    [(geom, 1) for geom in daegu_boundary.geometry],
    out_shape=rasterized.shape,
    transform=rio.transform.from_bounds(*daegu_boundary.total_bounds, prob_map.shape[1], prob_map.shape[0]),
    fill=0,
    all_touched=True,
    dtype=np.int16,
)

In [ ]:
plt.imshow(daegu_mask)

In [ ]:
# Rasterize the cropped Daegu boundary to use it as a mask
daegu_mask = features.rasterize(
    [(geom, 1) for geom in daegu_boundary.geometry],
    out_shape=rasterized.shape,
    transform=rio.transform.from_bounds(*daegu_boundary.total_bounds, prob_map.shape[1], prob_map.shape[0]),
    fill=0,
    all_touched=True,
    dtype=np.int16,
)

# Apply the mask to prob_map (exclude zero values)
prob_map_cropped = prob_map.copy()
prob_map_cropped[daegu_mask == 0] = np.nan

# Plot the cropped probability map
plt.imshow(prob_map_cropped, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Cropped Predictive Map for Daegu')
plt.show()

# Save the cropped probability map as a raster file
output_file = 'cropped_prob1_map.tif'
with rio.open(output_file, 'w', driver='GTiff', width=prob_map_cropped.shape[1], height=prob_map_cropped.shape[0], count=1, dtype=rio.float64, crs=rio.crs.CRS.from_epsg(4326), transform=rio.transform.from_bounds(*daegu_boundary.total_bounds, prob_map_cropped.shape[1], prob_map_cropped.shape[0])) as dst:
    dst.write(prob_map_cropped, 1)

## let's check linear regression 

In [ ]:
X = gdf_points[['ndvi']].dropna()  # Predictor
y = gdf_points['P_A_species'][gdf_points['ndvi'].notna()]  # Response
model = LinearRegression()
model.fit(X, y)

In [ ]:
# Predict probabilities over the entire area
ndvi_flat = ndvi.flatten()
ndvi_flat = ndvi_flat[~np.isnan(ndvi_flat)]  # Drop NaN values if any
# Predict using the linear regression model
predictions = model_ln.predict(ndvi_flat.reshape(-1, 1))
# Convert the predictions to probabilities using the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
probs = sigmoid(predictions)
# Create a probability map with the same shape as the original NDVI array
prob_map = np.full_like(ndvi, fill_value=np.nan)
# Assign the predicted probabilities to the corresponding positions in the probability map
prob_map.ravel()[~np.isnan(ndvi.ravel())] = probs
# Plot the probability map
plt.figure(figsize=(10, 8))
plt.imshow(prob_map, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Predictive Linear national Map of Probability')
plt.show()

In [ ]:
# Rasterize the cropped Daegu boundary to use it as a mask (linear)
daegu_mask = features.rasterize(
    [(geom, 1) for geom in daegu_boundary.geometry],
    out_shape=prob_map.shape,
    transform=rio.transform.from_bounds(*daegu_boundary.total_bounds, prob_map.shape[1], prob_map.shape[0]),
    fill=0,
    all_touched=True,
    dtype=np.int16,
)

# Apply the mask to prob_map (exclude zero values)
prob_map_cropped = prob_map.copy()
prob_map_cropped[daegu_mask == 0] = np.nan

# Plot the cropped probability map
plt.imshow(prob_map_cropped, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Cropped Predictive Map for Daegu')
plt.show()


## Below we can do excercise checking accuracy using training testing approach

In [ ]:
# let's check with test train data and ROC curve (A case for logistic regression )

In [ ]:
# we found there is only oe species that cross the thrreshold 
#o we use P__bengale data
# Prepare data for logistic regression
X = gdf_points[['ndvi']].dropna()  # Predictor
y = gdf_points['P_A_species'][gdf_points['ndvi'].notna()]  # Response
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)
# Fit logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)
# Predict probabilities on test set
y_prob = model.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Predict probabilities over the entire area
ndvi_flat = ndvi.flatten()
ndvi_flat = ndvi_flat[~np.isnan(ndvi_flat)]  # Drop NaN values if any
probs = model.predict_proba(ndvi_flat.reshape(-1, 1))[:, 1]
# Reshape probabilities to match the NDVI shape
prob_map = np.full_like(ndvi, fill_value=np.nan)
prob_map.ravel()[~np.isnan(ndvi.ravel())] = probs
# Plot probability map
plt.figure(figsize=(10, 8))
plt.imshow(prob_map, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Predictive Map with train_test approach of Probability')
plt.show()

In [ ]:
# Rasterize the cropped Daegu boundary to use it as a mask
daegu_mask = features.rasterize(
    [(geom, 1) for geom in daegu_boundary.geometry],
    out_shape=prob_map.shape,
    transform=rio.transform.from_bounds(*daegu_boundary.total_bounds, prob_map.shape[1], prob_map.shape[0]),
    fill=0,
    all_touched=True,
    dtype=np.int16,
)

# Apply the mask to prob_map (exclude zero values)
prob_map_cropped = prob_map.copy()
prob_map_cropped[daegu_mask == 0] = np.nan

# Plot the cropped probability map
plt.imshow(prob_map_cropped, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(label='Probability')
plt.title('Cropped Predictive Map for Daegu')
plt.show()

# Save the cropped probability map as a raster file
output_file = 'cropped_prob1_map.tif'
with rio.open(output_file, 'w', driver='GTiff', width=prob_map_cropped.shape[1], height=prob_map_cropped.shape[0], count=1, dtype=rio.float64, crs=rio.crs.CRS.from_epsg(4326), transform=rio.transform.from_bounds(*daegu_boundary.total_bounds, prob_map_cropped.shape[1], prob_map_cropped.shape[0])) as dst:
    dst.write(prob_map_cropped, 1)

In [ ]:
# A simple logistic regression code 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Generate random variables (features) and labels
np.random.seed(42)
X = np.random.rand(1000, 2)  # Example features (random variables)
y = (X.sum(axis=1) > 1.0).astype(int)  # Example labels based on a condition

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
